In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from tqdm import tqdm
import time
import random
import json

UserAgent - чтобы обходить блокировку краулера сайтом (спойлер: не поможет)

Сайт давал скачать примерно 10 страниц каждые полчаса, поэтому я не стала искать другой сайт или более сложные пути обхода, а просто собрала данные по частям

In [2]:
session = requests.session()

ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

Список из 4 шампуней, отзывы на которых возьмём

In [3]:
bad_reviews_links = []
good_reviews_links = []

shampoos = ['https://irecommend.ru/content/shampun-garnier-fructis-obem-i-vosstanovlenie-ekstralegkost-dlya-tonkikh-oslablennykh-volos',
           'https://irecommend.ru/content/shampun-nivea-shampun-ukhod-siyanie-i-obem',
           'https://irecommend.ru/content/shampun-ot-perkhoti-headshoulders-protiv-vypadeniya-volos',
           'https://irecommend.ru/content/shampun-pantene-pro-v-novaya-formula-gladkii-shelk']

Функция, находящая все ссылки на отзывы из страницы продукта и определяющая, хороший отзыв или плохой. Плохими считались отзывы от 3 звёзд и ниже, хорошими - только 5 звёзд. Отзывы на 4 звезды не рассматривались, так как в них, как правило, подробно описывают и положительные и отрицательные стороны, и такой отзыв сложно назвать только положительным или только отрицательным, поэтому они могут испортить выборку, и я не стала их брать.

In [4]:
def find_reviews(url):
    response = session.get(url, headers=headers)
    html_content = response.text.split('Смотрите также')[0]
    
    soup = BeautifulSoup(html_content, 'html.parser')
    news = soup.find_all('div', {'class': 'smTeaser woProduct plate reviews-list-item teaser-item'})
    
    links = []
    for new in news:
        stars = new.find('div', {'class': 'starsRating'})
        if stars:
            stars_num = len(str(stars).split('on')) - 1
            link = new.find('div', {'class': 'reviewTitle'})
            href = link.find('a').attrs['href']
            if stars_num < 4:
                bad_reviews_links.append(href)
            elif stars_num > 4:
                good_reviews_links.append(href)
    return links

In [5]:
for link in shampoos:
    find_reviews(link)
    time.sleep(random.uniform(1.1, 5.2))
print(len(good_reviews_links), len(bad_reviews_links))

56 59


(это списки из работы кода выше, но из-за блокировок вместо повторного запуска кода загрузим сохранённые)

In [4]:
bad_reviews_links = ['/content/sploshnoe-vrane-foto-obeshchanii-i-emotsii', '/content/i-gde-zhe-moi-obem-i-vosstanovlenie', '/content/ne-sdelal-volosy-obemnee-ne-vosstanovil-ikh-zato-ulybnul-bredovymi-instrumentalnymi-testami', '/content/obem-moim-tonkim-dlinnym-volosam-deistvitelno-pridal-foto-posle-ispolzovaniya', '/content/ispolzovala-dlya-smyvaniya-maslyanykh-masok-i-prosto-na-kazhdyi-den-moe-mnenie', '/content/legkost-volos-mozhet-byt-vse-zhe-pushistost-eshche-nepromytost', '/content/ekhkak-ya-lyublyu-zapakh-etogo-shampunya-zhal-no-posle-takogo-pridetsya-perestat-im-polzovat', '/content/ne-dlya-moikh-volos-navernoe-khotya-ne-znayu-dazhefoto-posle-mytya', '/content/razocharovanie-528', '/content/vosstanavlivayu-im-poly', '/content/intuitsiya-ne-podvela-1', '/content/snachala-khoroshopotom-uzhasno', '/content/topolinyi-pukh-zhara-iyun-eto-ya-pro-svoi-volosy-posle-mytya-shampunem-nivea-siyanie-i-obem', '/content/shampun-etot-ochen-strannyi-predmet-volosy-chistye-golova-net', '/content/vmesto-brilliantovogo-bleska-i-obema-poluchila-zhestkie-volosy-obem-i-blesk-zabyli-nalit-vo', '/content/dva-minusa-kotorye-zastavili-menya-prekratit-ispolzovanie-dannogo-shampunya', '/content/kraine-nepriyatnyi-effekt-ot-shampunya', '/content/neplokhoi-shampun-no-minus-est-dlya-moikh-volos-ogromnyi', '/content/ziyayushchii-proval-obeshchannogo-siyaniya', '/content/deistvitelno-li-ukhod-i-siyanie', '/content/ot-perkhoti-da-eshche-i-ot-vypadeniya-tak-li-effektiven-pirition-tsinka-kak-nam-pytayutsya-v', '/content/dazhe-nogi-im-myt-ne-stanu-podrobno-rasskazhu-pochemu', '/content/dazhe-i-ne-dumaite-pokupat', '/content/khotite-prekrasnye-issushennye-volosy-i-zhirnye-korni-na-sleduyushchii-dentogdavam-opredelen', '/content/ocherednoe-razocharovanie-veka-chuda-ne-proizoshlo', '/content/net-razmyshleniya-o-marketingovykh-khodakh-odnogo-strannogo-cheloveka', '/content/momentalnyi-effekt-gryaznykh-volos', '/content/budte-ostorozhny-69', '/content/ne-shampun-razocharovanie-0', '/content/ni-ot-perkhoti-ni-ot-vypadeniya-volos-ne-pomogaet', '/content/tak-mnogo-minusov-tak-malo-plyusov', '/content/volosy-stali-vypadat-eshche-bolshe-0', '/content/strannyi-effekt-ubral-perkhot-i-vysushil-i-bez-togo-sukhie-volosy-i-vse-eto-za-odno-primenen', '/content/shampun-isportilsya', '/content/ne-rekomenduyu-1579', '/content/otlichnyi-shampun-432', '/content/vyzyvaet-perkhot-3', '/content/proizvoditeli-opechatalis-ne-protiv-perkhoti-dlya-perkhoti', '/content/chem-dorozhe-tem-kruche-reklama', '/content/ne-ot-vypadeniya-dlya-vypadeniya', '/content/shampun-kak-shampun-5', '/content/uzhasnyi-shampun-96', '/content/ekhpantinnu-ne-sudba-tak-ne-sudbaotzyv-iz-razryadaezhiki-plakalikololisno-prodolzhali-est-ka', '/content/yavno-ne-dlya-chastogo-ispolzovaniya-foto-volos', '/content/ves-lob-v-pryshchakh-foto-shampunya', '/content/pakli-na-golove-byli-pakli-i-ostalis-shampun-ne-sdelal-nichego', '/content/ne-dumala-chto-shelk-vyglyadit-tak', '/content/volosy-neplokho-vyglyadyat-posle-etogo-shampunyano-k-sozhaleniyu-u-nego-uzhasnyi-khimicheski', '/content/vse-chto-ugodno-tolko-ne-shelk', '/content/nachali-lezt-volosy-0', '/content/poyavilas-perkhot-13', '/content/shampun-etoi-serii-ne-vpechatlil-balzam-super', '/content/uzhasnyikak-vse-serii-pantene-pro-v', '/content/moi-verdikt', '/content/okh-i-nadoeli-oni-s-etimi-novymi-formulami', '/content/ostavlyaet-zhelat-luchshego-15', '/content/shelka-chto-ne-vidno', '/content/gladkii-shelk-nesootvetstvie-kakoe', '/content/poyavilas-perkhot-2']

In [5]:
good_reviews_links = ['/content/i-vse-ravno-periodicheski-ruki-tyanutsya-kupit-znamenituyu-zelenuyu-butylochku-ot-garnier-bl', '/content/prekrasnyi-shampun-dlya-tonkikh-i-oslablennykh-volos', '/content/ya-nashla-idealnyi-dlya-sebya-shampun', '/content/poprobuyu-ispolzovat-regulyarnofoto-shampunya-i-kollektsii-garnier', '/content/podoidet-dlya-tonkikh-oslablennykh-volos-foto', '/content/zamechatelnyi-shampun-49', '/content/sukhie-volosy', '/content/blestyashchii-lyubimchik-s-almaznoi-pudroi-kotoromu-moi-volosy-rady-uzhe-3-goda', '/content/myagkii-i-pushistyi-kak-popa-khomyaka', '/content/teper-v-spiske-lyubimchikov-foto-rezultata', '/content/shampun-ot-nivea-vytesnil-s-polki-moego-prezhnego-favorita', '/content/neozhidanno-khoroshii-shampun-1', '/content/pridaet-volosam-shelkovistost', '/content/idealnoe-sredstvo-dlya-moikh-volos-1', '/content/luchshe-lyuksa-3', '/content/normalnyi-shampun-49', '/content/volosy-lezut-klokami-i-vyzyvayut-paniku-shampun-headshoulders-obeshchaet-nam-bystroe-i-nadez', '/content/zamechatelnyi-shampun-pomogaet-ot-perkhoti-foto-volos-v-otzyve', '/content/100-izbavlenie-ot-perkhoti-za-odin-den-bez-vreda-volosam-rekomenduyu-foto', '/content/vpervye-shampun-headshoulders-mne-deistvitelno-pomog-izbavitsya-ot-perkhoti-foto-volos-sosta', '/content/povyshu-obshchii-reiting-etogo-shampunya-mne-ochen-ponravilsya-ne-samyi-plokhoi-shampun-iz-v', '/content/luchshii-shampun-43', '/content/moi-lyubimyi-shampun-47', '/content/odin-iz-lyubimchikov-foto', '/content/mne-ochen-ponravilsya-177', '/content/moe-lichnoe-mnenie-o-shampune-headshoulders', '/content/moi-geroi-1', '/content/pomogaet-348', '/content/khoroshii-shampuns-ochen-priyatnym-aromatom', '/content/otlichnyi-shampun-rekomenduyu-0', '/content/super-414', '/content/odin-iz-nemnogikh-shampunei-kotoryi-mne-idealno-podkhodit', '/content/moi-volosy-vlyubleny-okonchatelno-i-bespovorotno-mnogo-foto-rezultata-ispolzovaniya-neskolki', '/content/moi-volosy-naglyadno-rasskazhut-vam-podoshel-li-im-shampun-pantene-pro-v-gladkii-shelk-foto', '/content/u-menya-svoya-formula-i-volosy-kak-brilliant-foto-dokazatelstva', '/content/mne-bolshe-ne-nuzhen-utyuzhok-foto-do-i-posle', '/content/balzam-gladkii-shelk-chto-poteryano-v-obeme-vospolneno-v-kachestve-volos-foto', '/content/idealno-ochishchaet-volosy-dazhe-ot-masel-dlya-volos-chto-zhe-eshche-delaet-s-volosami', '/content/udivitelno-chto-tak-malo-otzyvov-na-balzam-pantene-pro-v-gladkii-shelk', '/content/shampun-ochen-poradoval-foto-vnutri', '/content/ne-poimu-poddelka-ili-raznye-sostavy-foto', '/content/idealen-dlya-vyushchikhsya-volos-idealen-dlya-menya', '/content/gladkie-pryamye-poslushnye-volosy', '/content/dlya-menya-idealenfoto-volos-posle-primeneniya', '/content/volosy-takie-gladkie-chto-khochetsya-k-nim-prikasatsya', '/content/kachestvo-shampunya-poradovalo-menya-i-moi-volosy', '/content/ochen-dazhe-nichego-258', '/content/luchshii-iz-vsekh-shampunei-chto-ya-ispolzovala', '/content/idealnoe-sredstvo-ya-vlyubilas-v-nego', '/content/nakonets-ya-nashlo-ego', '/content/na-dannyi-moment-best', '/content/otlichnyi-shampun-171', '/content/kupila-pantene-i-ne-razocharovalas', '/content/eto-klassika-2', '/content/otlichnyi-shampun-118', '/content/otlichnyi-shampun-110']

выберем отзывы из разных мест списка, чтобы выборка была более сбалансированной, с разными товарами

In [6]:
bad_check = []
good_check = []
for i in range(0, 56, 8):
    bad_check.append(bad_reviews_links[i])
    good_check.append(good_reviews_links[i])
for link in bad_check:
    bad_reviews_links.remove(link)
for link in good_check:
    good_reviews_links.remove(link)

In [7]:
print(len(bad_check), len(good_check))
print(len(good_reviews_links), len(bad_reviews_links))

7 7
49 52


In [8]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
morph = MorphAnalyzer()

Функция, которая выделяет из страницы с отзывом текст, токенизирует его, приводит к нижнему регистру и начальной форме

In [9]:
def parse_review(url):
    headers = {'User-Agent': ua.random}
    response = session.get(url, headers=headers)
    html_content = response.text.split('сообщить о нарушении')[0]
    html_content = html_content.split('Опубликовано')[-1]
    html_content = html_content.split('Достоинства')[0]
    
    soup = BeautifulSoup(html_content, 'html.parser')
    text = soup.text
    
    tokens = tokenizer.tokenize(text)
    lemmas = []
    for token in tokens:
        token = token.lower()
        analyze = morph.parse(token)[0]
        lemmas.append(analyze.normal_form)
    return lemmas

In [10]:
bad_words = []
good_words = []

Из-за блокировки загрузим уже обработанное раньше

In [11]:
with open('bad_words.json', encoding="utf-8") as file:
    bad_words = json.load(file)

In [12]:
bad_words[:100]

['16',
 'сентябрь',
 '2018',
 '20',
 '24',
 'и',
 'где',
 'же',
 'мой',
 'объём',
 'и',
 'восстановление',
 'добрый',
 'день',
 'ну',
 'или',
 'вечер',
 'сегодня',
 'хотеть',
 'поделиться',
 'свой',
 'впечатление',
 'о',
 'свой',
 'недавний',
 'приобретение',
 'а',
 'именно',
 'шампунь',
 'fructis',
 'укреплять',
 'объём',
 'и',
 'восстановление',
 'этот',
 'яркий',
 'флакон',
 'сразу',
 'броситься',
 'я',
 'в',
 'глаз',
 'и',
 'показаться',
 'очень',
 'необычный',
 'на',
 'фон',
 'остальной',
 'флаконпереть',
 'покупка',
 'я',
 'не',
 'обратить',
 'внимание',
 'на',
 'крышка',
 'и',
 'за',
 'это',
 'я',
 'ругать',
 'себя',
 'каждый',
 'раз',
 'когда',
 'мой',
 'голова',
 'это',
 'шампунь',
 'крышечка',
 'ужасно',
 'тугой',
 'я',
 'думать',
 'что',
 'с',
 'время',
 'она',
 'разработаться',
 'и',
 'стать',
 'мягкий',
 'открываться',
 'и',
 'закрываться',
 'ан',
 'нет',
 'обещание',
 'производитель',
 'а',
 'теперь',
 'поделиться',
 'свой',
 'впечатление',
 'и',
 'если',
 'честно',
 'не'

In [64]:
for link in tqdm(bad_reviews_links[:30]):
    lemmas = parse_review('https://irecommend.ru' + link)
    print(lemmas[:50])
    bad_words.extend(lemmas)
    time.sleep(random.uniform(1.1, 5.2))

  0%|                                                                                           | 0/15 [00:00<?, ?it/s]

['24', 'январь', '2015', '15', '32', 'хотеть', 'прекрасный', 'иссушить', 'волос', 'и', 'жирный', 'корень', 'на', 'следующий', 'день', 'тогда', 'вы', 'определённо', 'нужный', 'он', 'вызывать', 'перхоть', 'после', 'первый', 'применение', 'фото', 'здравствуйте', 'красавица', 'сегодня', 'я', 'поведать', 'вы', 'о', 'шампунь', 'head', 'shoulders', 'против', 'выпадение', 'волос', 'для', 'женщина', 'с', 'это', 'шампунь', 'я', 'прийтись', 'познакомиться', 'так', 'как', 'я']


  7%|█████▌                                                                             | 1/15 [00:04<01:09,  4.99s/it]

['3', 'май', '2012', '20', '32', 'нет', 'размышление', 'о', 'маркетинговый', 'ход', 'один', 'странный', 'человек', 'мама', 'любить', 'шампунь', 'этот', 'марка', 'покупать', 'постоянно', 'только', 'они', 'я', 'же', 'достаточно', 'сомнительно', 'относиться', 'к', 'подобный', 'средство', 'где', 'обещать', 'что', 'говориться', 'стопицот', 'в', 'один', 'и', 'увлажнение', 'ты', 'и', 'питание', 'и', 'от', 'перхоть', 'избавлять', 'и', 'мужик', 'привлекать', 'и']


 13%|███████████                                                                        | 2/15 [00:07<00:56,  4.32s/it]

...

 87%|███████████████████████████████████████████████████████████████████████           | 13/15 [00:54<00:07,  3.97s/it]

[]


 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [00:58<00:03,  3.93s/it]

[]


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [01:00<00:00,  4.03s/it]


Сохраняем, когда сайт опять начал блокировать

In [13]:
with open('bad_words.json', 'w', encoding="utf-8") as outfile:
    json.dump(bad_words, outfile)

In [143]:
bad_reviews_links[29]

'/content/ne-rekomenduyu-1579'

аналогично с хорошими словами

In [14]:
with open('good_words.json', encoding="utf-8") as file:
    good_words = json.load(file)

In [149]:
for link in tqdm(good_reviews_links[:30]):
    lemmas = parse_review('https://irecommend.ru' + link)
    print(lemmas[:50])
    good_words.extend(lemmas)
    time.sleep(random.uniform(1.1, 5.2))

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

['5', 'март', '2013', '12', '40', 'отличный', 'шампунь', 'рекомендовать', 'шампунь', 'head', 'shoulders', 'против', 'выпадение', 'волос', 'пользоваться', 'давно', 'периодически', 'сменять', 'он', 'другой', 'шампунь', 'этот', 'же', 'марка', 'волос', 'действительно', 'выпадать', 'маленький', 'но', 'эффект', 'есть', 'только', 'если', 'регулярно', 'использовать', 'этот', 'шампунь', 'стоить', 'только', 'сменить', 'он', 'на', 'другой', 'как', 'волос', 'начинать', 'выпадать', 'как', 'и', 'до']


 20%|████████████████▊                                                                   | 1/5 [00:03<00:12,  3.01s/it]

['7', 'октябрь', '2010', '13', '20', 'супер', 'в', 'один', 'время', 'очень', 'сильно', 'начать', 'выпадать', 'волос', 'я', 'стать', 'страшно', 'такой', 'количество', 'волос', 'в', 'день', 'терять', 'просто', 'не', 'мыслить', 'достаточно', 'быть', 'просто', 'провести', 'рука', 'по', 'волос', 'и', 'наблюдать', 'как', 'пар', 'десяток', 'волос', 'красоваться', 'уже', 'у', 'ты', 'на', 'ладонь', 'я', 'не', 'говорить', 'даже', 'о']


 40%|█████████████████████████████████▌                                                  | 2/5 [00:06<00:09,  3.13s/it]

...

 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:08<00:05,  2.84s/it]

['5', 'октябрь', '2015', '20', '35', 'мой', 'волос', 'наглядно', 'рассказать', 'вы', 'подойти', 'ли', 'они', 'шампунь', 'pantene', 'pro', 'v', 'гладкий', 'шёлк', 'фото', 'приветствовать', 'весь', 'читатель', 'мой', 'отзыв', 'наверняка', 'всё', 'вы', 'знакомый', 'с', 'шампунь', 'pantene', 'pro', 'v', 'и', 'сегодня', 'я', 'хотеть', 'поделиться', 'свой', 'опыт', 'применение', 'шампунь', 'pantene', 'pro', 'v', 'гладкий', 'шёлк', 'знакомый', 'с']


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:11<00:02,  2.77s/it]

['22', 'октябрь', '2012', '22', '20', 'у', 'я', 'свой', 'формула', 'и', 'волос', 'как', 'бриллиант', 'фото', 'доказательство', 'согласиться', 'что', 'ошибиться', 'с', 'шампунь', 'не', 'такой', 'уж', 'и', 'беда', 'конечно', 'если', 'вызвать', 'раздражение', 'не', 'привести', 'к', 'серьёзный', 'проблема', 'с', 'здоровье', 'стоить', 'не', 'много', 'если', 'что', 'то', 'не', 'устраивать', 'легко', 'поменять', 'без', 'последствие', 'для', 'здоровье']


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:13<00:00,  2.70s/it]


In [15]:
with open('good_words.json', 'w', encoding="utf-8") as outfile:
    json.dump(good_words, outfile)

In [16]:
good_words[:100]

['4',
 'декабрь',
 '2016',
 '14',
 '00',
 'прекрасный',
 'шампунь',
 'для',
 'тонкий',
 'и',
 'ослабить',
 'волос',
 'я',
 'всегда',
 'покупать',
 'разный',
 'шампунь',
 'чтобы',
 'волос',
 'не',
 'привыкать',
 'к',
 'определённый',
 'ингредиент',
 'поэтому',
 'шампунь',
 'я',
 'перепробывать',
 'немало',
 'если',
 'бы',
 'я',
 'покупать',
 'каждый',
 'раз',
 'один',
 'и',
 'тот',
 'же',
 'шампунь',
 'то',
 'данный',
 'шампунь',
 'мочь',
 'бы',
 'претендовать',
 'на',
 'это',
 'место',
 'хотеть',
 'сказать',
 'что',
 'в',
 'это',
 'шампунь',
 'нет',
 'ничто',
 'особенный',
 'но',
 'как',
 'ни',
 'странно',
 'это',
 'я',
 'и',
 'нравиться',
 'я',
 'пробовать',
 'разный',
 'шампунь',
 'в',
 'многий',
 'быть',
 'куча',
 'плюс',
 'но',
 'какой',
 'то',
 'зачастую',
 'небольшой',
 'минус',
 'всё',
 'портить',
 'у',
 'один',
 'раздражать',
 'запах',
 'другой',
 'пересушивать',
 'волос',
 'после',
 'другой',
 'волос',
 'быстро',
 'становиться',
 'грязный',
 'и',
 'так',
 'далее',
 'то']

In [94]:
good_reviews_links[24]

'/content/khoroshii-shampuns-ochen-priyatnym-aromatom'

Теперь множества с только положительными и только отрицательными словами

In [17]:
only_bad = set(bad_words).difference(set(good_words))
only_good = set(good_words).difference(set(bad_words))

In [18]:
only_bad

{'02',
 '05',
 '07',
 '120',
 '160',
 '180',
 '200',
 '29',
 '2в1',
 '32',
 '34',
 '340р',
 '350',
 '400мл',
 '4назад',
 '51',
 '54',
 '69',
 '6ить',
 '74',
 '82193',
 '821930986050',
 '90',
 '98',
 'and',
 'benzyl',
 'black',
 'butylphenyl',
 'camellia',
 'cane',
 'carbomer',
 'chiharu',
 'citrus',
 'cocamide',
 'cocoamphodiacetate',
 'colorич',
 'content',
 'denat',
 'dercosпилинг',
 'disodium',
 'distearate',
 'edelweissич',
 'elisabetич',
 'eto',
 'extract',
 'fiberbond',
 'fix',
 'fragrance',
 'fruit',
 'ginseng',
 'glycol',
 'grapefruit',
 'hcl',
 'hexylene',
 'http',
 'hydroxide',
 'hydroxypropyl',
 'i',
 'in',
 'irecommend',
 'isopropanolamine',
 'kelualалеран',
 'kerastaseидея',
 'lauryl',
 'leaf',
 'lebel',
 'lemon',
 'limonum',
 'loversич',
 'magnesium',
 'malus',
 'medica',
 'methylpropional',
 'mipa',
 'mlгде',
 'moe',
 'nakonets',
 'niacinamide',
 'octadecanediol',
 'officinarum',
 'oksana76ruич',
 'oleamido',
 'oleth',
 'panax',
 'peel',
 'podruzh',
 'priceшикарный',
 'p

In [19]:
only_good

{'0',
 '00',
 '04',
 '09',
 '109',
 '119',
 '129',
 '165',
 '168',
 '170',
 '1911',
 '200мл',
 '2010',
 '2021',
 '239',
 '250мл',
 '2х',
 '300',
 '329',
 '35',
 '36',
 '360',
 '38',
 '3х',
 '40',
 '42',
 '46',
 '47',
 '53',
 '56',
 '75',
 '9',
 '90руб',
 '92',
 '95',
 'ag',
 'airtouch',
 'beiersdorf',
 'betaine',
 'care',
 'castor',
 'cocamidopropyl',
 'cristalsyoss',
 'diamond',
 'doux',
 'dr',
 'elseve',
 'essence',
 'express',
 'gliss',
 'gloss',
 'glossing',
 'green',
 'hair',
 'hs',
 'hydrogenated',
 'kur',
 'l',
 'lanolin',
 'limonene',
 'macadamia',
 'maybelline',
 'naturals',
 'nutri',
 'oil',
 'olia',
 'ollin',
 'ombres',
 'oreal',
 'p',
 'palette',
 'palmer',
 'pantene',
 'peg',
 'perfect',
 'ph',
 'powder',
 'preference',
 'pro',
 'refreshing',
 'sante',
 'seal',
 'seed',
 'shine',
 'sholders',
 'shouiders',
 'shoulers',
 'sles',
 'superdrug',
 'superlady',
 'tea',
 'ternifolia',
 'treehead',
 'ultra',
 'v',
 'volume',
 'абразивный',
 'аварийный',
 'авокадо',
 'агент',
 'агр

фильтрация по количеству вхождений

In [20]:
to_delete = set()
for word in only_bad:
    if bad_words.count(word) < 2:
        to_delete.add(word)

In [21]:
bad = only_bad-to_delete

In [22]:
bad

{'02',
 '07',
 '32',
 '34',
 '400мл',
 '51',
 '54',
 'benzyl',
 'butylphenyl',
 'camellia',
 'cane',
 'carbomer',
 'citrus',
 'cocamide',
 'cocoamphodiacetate',
 'content',
 'denat',
 'disodium',
 'distearate',
 'extract',
 'fragrance',
 'fruit',
 'ginseng',
 'glycol',
 'hcl',
 'hexylene',
 'http',
 'hydroxide',
 'hydroxypropyl',
 'irecommend',
 'isopropanolamine',
 'leaf',
 'lemon',
 'limonum',
 'magnesium',
 'malus',
 'medica',
 'methylpropional',
 'mipa',
 'niacinamide',
 'octadecanediol',
 'officinarum',
 'oleamido',
 'oleth',
 'panax',
 'peel',
 'propylene',
 'pyridoxine',
 'pyrus',
 'root',
 'ru',
 'saccharum',
 'salicylic',
 'sinensis',
 'sugar',
 'water',
 'бережный',
 'беспроблемный',
 'биоактивный',
 'близкий',
 'везде',
 'вечером',
 'восстанавливать',
 'восстановить',
 'впечатлить',
 'выветриваться',
 'выделить',
 'выкинуть',
 'выливать',
 'высохнуть',
 'высушить',
 'где',
 'говориться',
 'грязниться',
 'дизайн',
 'добавлять',
 'домой',
 'дописать',
 'допользовать',
 'ежедне

In [23]:
to_delete = set()
for word in only_good:
    if good_words.count(word) < 2:
        to_delete.add(word)

In [24]:
good = only_good-to_delete

In [25]:
good

{'165',
 '250мл',
 '300',
 '40',
 'beiersdorf',
 'betaine',
 'care',
 'castor',
 'cocamidopropyl',
 'diamond',
 'doux',
 'elseve',
 'gliss',
 'hs',
 'hydrogenated',
 'kur',
 'lanolin',
 'limonene',
 'macadamia',
 'oil',
 'oreal',
 'pantene',
 'peg',
 'ph',
 'powder',
 'pro',
 'seed',
 'sles',
 'ternifolia',
 'ultra',
 'v',
 'агрессивный',
 'аллергический',
 'алмазный',
 'антиоксидант',
 'антисептик',
 'антистатик',
 'аптека',
 'ароматизатор',
 'беспокоить',
 'блестеть',
 'блонда',
 'болячка',
 'бороться',
 'брызкай',
 'важный',
 'верить',
 'вещество',
 'влага',
 'влиять',
 'вода',
 'воздушный',
 'возникать',
 'возникнуть',
 'волосы5',
 'восторг',
 'впервые',
 'вредный',
 'всерекомендовать',
 'вспенить',
 'всякий',
 'вызвать',
 'выпускать',
 'высыхание',
 'выше',
 'вязкость',
 'глупо',
 'годность',
 'гормональный',
 'дальнейший',
 'далёкий',
 'данные',
 'дарить',
 'добиться',
 'достаточный',
 'достать',
 'енд',
 'жаловаться',
 'желтизна',
 'жизненный',
 'жирнит',
 'забота',
 'загустител

Функция, аналогичная разбору сайта с отзывом, только для текстового формата (поскольку отзывы на проверку я сохранила так)

In [27]:
def parse_review_text(text):
    tokens = tokenizer.tokenize(text)
    lemmas = []
    for token in tokens:
        token = token.lower()
        analyze = morph.parse(token)[0]
        lemmas.append(analyze.normal_form)
    return lemmas

функция, угадывающая, является ли отзыв хорошим

In [42]:
def if_good(lemmas):
    bad_count = 0
    good_count = 0
    for lemma in lemmas:
        if lemma in bad:
            bad_count += 1
        if lemma in good:
            good_count += 1
    print(bad_count, good_count)
    if bad_count > good_count:
        print('bad')
        return 'bad'
    else:
        print('good')
        return 'good'

In [43]:
tp = 0
tn = 0
fp = 0
fn = 0

In [44]:
with open('bad_check.txt', encoding = 'utf-8') as file:
    text = file.read()
    reviews = text.split('179179179')

In [45]:
for review in reviews:
    ans = if_good(parse_review_text(review))
    if ans == 'bad':
        tn += 1
    else:
        fp +=1

8 4
bad
0 4
good
11 2
bad
7 11
good
11 3
bad
0 4
good
1 5
good


In [46]:
with open('good_check.txt', encoding = 'utf-8') as file:
    text = file.read()
    reviews = text.split('179179179')

In [47]:
for review in reviews:
    ans = if_good(parse_review_text(review))
    if ans == 'good':
        tp += 1
    else:
        fn += 1

4 5
good
10 14
good
22 27
good
3 1
bad
15 13
bad
2 9
good
0 3
good


In [48]:
accuracy = (tn+tp)/(tn+tp+fn+fp)
print(accuracy)

0.5714285714285714


Как можно улучшить:
- смотреть не на "встречается только в отрицательных отзывах", а на долю в положительных и отрицательных отзывах, потому что даже слово "плохой" может быть в положительном отзыве, но всё же во много раз реже, чем в отрицательном
- смотреть не на токены, а на биграммы, чтобы отлавливать словосочетания с "не" и "но"